# LSTM Forecasting Model

O'Reilly Machine Learning and Security
* https://github.com/oreilly-mlsec/mlsec.net

Chapter 3 Resources
* https://github.com/oreilly-mlsec/book-resources/blob/master/chapter3/lstm-anomaly-detection.ipynb

In [6]:
import numpy as np

import tensorflow as tf
from collections import deque
from featurizer import BasicFeaturizer
from utils import *
import pickle

## Data Loading and Standardization

In [12]:
print("loading data...")
data = pickle.load(open('../simulation/small-data1.pkl')) # use a file generated by simulation/dataset.py

fr = BasicFeaturizer()
packets = np.array([fr.featurize(packet) for packet in data])

print("standardizing data... (impossible given protocol is invariable)")
means = np.apply_along_axis(np.mean, 0, packets)
stds = np.apply_along_axis(np.std, 0, packets)
packets -= means
# packets /= stds

print("done.")

# TODO: cull non-continuous variables OR manually define loss

loading data...
standardizing data... (impossible given protocol is invariable)
done.


## Hyper-parameters

In [14]:
epochs = 10
batch_size = 50
sequence_length = 4
features = len(packets[0])
mean_window = 40
loss_tolerance = 2
train_test_split = 8

## Class Definition

In [15]:
class ForecastModel(object):
    
    def __init__(self):
        self.model = self.generate_model()
    
    def generate_model(self):
        
        layers = [
            tf.keras.layers.LSTM(input_shape=(sequence_length - 1, features), 
                                 units=32, 
                                 return_sequences=True),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.LSTM(units=128,
                                 return_sequences=True),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.LSTM(units=100,
                                 return_sequences=False),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(units=features),
            tf.keras.layers.Activation('linear')
        ]
        model = tf.keras.Sequential(layers)
        model.compile(loss='mean_squared_error', optimizer='rmsprop')
        
        return model
    
    def prepare_data(self, data, training=True):
        
        if training:

            print('creating train n-grams...')

            train_grams = []
            for i in range(0, len(data) - sequence_length):
                train_grams.append(data[i: i + sequence_length])
            train_grams = np.array(train_grams)

            print('train data shape : ', train_grams.shape)

            self.x_train = train_grams[:, :-1]
            self.y_train = train_grams[:, -1]

        else:

            print('creating test n-grams...')

            test_grams = []
            for i in range(0, len(data) - sequence_length):
                test_grams.append(data[i: i + sequence_length])
            test_grams = np.array(test_grams)

            print('test data shape : ', test_grams.shape)  

            self.x_test = test_grams[:, :-1]
            self.y_test = test_grams[:, -1]        
    
    def train(self, data):
        
        self.prepare_data(data)
        self.train_history = self.model.fit(self.x_train, self.y_train,
                                            batch_size=batch_size, epochs=epochs)
        
    def test(self, data):
        
        assert self.train_history is not None 
        
        self.prepare_data(data, training=False)
        
        losses_in_window = deque()
        moving_mean = 0
        rolling_std = 0
        
        for i in range(self.x_test.shape[0]):
            
            test_loss = self.model.evaluate(x=np.expand_dims(self.x_test[i], 0), 
                                            y=np.expand_dims(self.y_test[i], 0), batch_size=1)
            
            if i < mean_window:
                moving_mean += test_loss / mean_window
                losses_in_window.append(test_loss)
                print("build mean")
            else:
                moving_mean += (test_loss - losses_in_window[0]) / mean_window
                losses_in_window.popleft()
                losses_in_window.append(test_loss)
                
                # not efficient
                rolling_std = np.std(losses_in_window)
                
                if np.abs(test_loss - moving_mean) < rolling_std * loss_tolerance:
                    print("all clear")
                else:
                    print("\nMALICIOUS\n")

In [16]:
model = ForecastModel()

split = (len(packets) * train_test_split) // 10

model.train(packets[:split])
model.test(packets[split:])

creating train n-grams...
('train data shape : ', (1596, 4, 22))
Epoch 1/10
1596/1596 [==============================] - 2s 2ms/step - loss: 130728729713332784.0000
Epoch 2/10
1596/1596 [==============================] - 0s 236us/step - loss: 130728731123459872.0000
Epoch 3/10
1596/1596 [==============================] - 0s 263us/step - loss: 130728730036262640.0000
Epoch 4/10
1596/1596 [==============================] - 0s 234us/step - loss: 130728728712250160.0000
Epoch 5/10
1596/1596 [==============================] - 0s 230us/step - loss: 130728729917855024.0000
Epoch 6/10
1596/1596 [==============================] - 0s 249us/step - loss: 130728730531421776.0000
Epoch 7/10
1596/1596 [==============================] - 0s 247us/step - loss: 130728731866198576.0000
Epoch 8/10
1596/1596 [==============================] - 1s 336us/step - loss: 130728729777918752.0000
Epoch 9/10
1596/1596 [==============================] - 0s 247us/step - loss: 130728728540020896.0000
Epoch 10/10
1596/15

all clear
1/1 [==============================] - 0s 8ms/step
all clear
1/1 [==============================] - 0s 5ms/step
all clear
1/1 [==============================] - 0s 6ms/step
all clear
1/1 [==============================] - 0s 3ms/step
all clear
1/1 [==============================] - 0s 3ms/step
all clear
1/1 [==============================] - 0s 3ms/step
all clear
1/1 [==============================] - 0s 3ms/step
all clear
1/1 [==============================] - 0s 3ms/step
all clear
1/1 [==============================] - 0s 3ms/step

MALICIOUS

1/1 [==============================] - 0s 3ms/step

MALICIOUS

1/1 [==============================] - 0s 3ms/step
all clear
1/1 [==============================] - 0s 4ms/step
all clear
1/1 [==============================] - 0s 4ms/step
all clear
1/1 [==============================] - 0s 4ms/step
all clear
1/1 [==============================] - 0s 3ms/step
all clear
1/1 [==============================] - 0s 4ms/step
all clear
1/1 [=====

1/1 [==============================] - 0s 3ms/step

MALICIOUS

1/1 [==============================] - 0s 3ms/step

MALICIOUS

1/1 [==============================] - 0s 3ms/step
all clear
1/1 [==============================] - 0s 3ms/step
all clear
1/1 [==============================] - 0s 3ms/step
all clear
1/1 [==============================] - 0s 3ms/step
all clear
1/1 [==============================] - 0s 3ms/step
all clear
1/1 [==============================] - 0s 3ms/step
all clear
1/1 [==============================] - 0s 3ms/step
all clear
1/1 [==============================] - 0s 3ms/step
all clear
1/1 [==============================] - 0s 3ms/step
all clear
1/1 [==============================] - 0s 3ms/step
all clear
1/1 [==============================] - 0s 3ms/step
all clear
1/1 [==============================] - 0s 3ms/step
all clear
1/1 [==============================] - 0s 3ms/step
all clear
1/1 [==============================] - 0s 3ms/step
all clear
1/1 [===============

all clear
1/1 [==============================] - 0s 3ms/step
all clear
1/1 [==============================] - 0s 3ms/step
all clear
1/1 [==============================] - 0s 3ms/step
all clear
1/1 [==============================] - 0s 3ms/step
all clear
1/1 [==============================] - 0s 2ms/step
all clear
1/1 [==============================] - 0s 3ms/step

MALICIOUS

1/1 [==============================] - 0s 3ms/step
all clear
1/1 [==============================] - 0s 3ms/step
all clear
1/1 [==============================] - 0s 4ms/step
all clear
1/1 [==============================] - 0s 3ms/step
all clear
1/1 [==============================] - 0s 3ms/step
all clear
1/1 [==============================] - 0s 3ms/step
all clear


In [17]:
model.train_history